In [36]:
!cat ../fib.py

import click

def fibonacci(N):
    if N < 2:
        return N

    return fibonacci(N - 2) + fibonacci(N - 1)

@click.command()
@click.option("-l", default=False, is_flag=True)
@click.argument('n', type=click.INT)
def main(n, l):
    if l:
        for i in range(n):
            print("The fibonacci number for '{}' is {}".format(
                i, fibonacci(i)))

    print("The fibonacci number for '{}' is {}".format(
        n, fibonacci(n)))

if __name__ == "__main__":
    main()


In [37]:
from fib import fibonacci
fibonacci(32)

2178309

In [38]:
!fib 32

The fibonacci number for '32' is 2178309


In [39]:
!fib -l 20

The fibonacci number for '0' is 0
The fibonacci number for '1' is 1
The fibonacci number for '2' is 1
The fibonacci number for '3' is 2
The fibonacci number for '4' is 3
The fibonacci number for '5' is 5
The fibonacci number for '6' is 8
The fibonacci number for '7' is 13
The fibonacci number for '8' is 21
The fibonacci number for '9' is 34
The fibonacci number for '10' is 55
The fibonacci number for '11' is 89
The fibonacci number for '12' is 144
The fibonacci number for '13' is 233
The fibonacci number for '14' is 377
The fibonacci number for '15' is 610
The fibonacci number for '16' is 987
The fibonacci number for '17' is 1597
The fibonacci number for '18' is 2584
The fibonacci number for '19' is 4181
The fibonacci number for '20' is 6765


## Subprocess - a poor man's multiprocessing

The [subprocess](https://docs.python.org/3/library/subprocess.html#module-subprocess) module allows you to spawn new processes, connect to their input/output/error pipes, and obtain their return codes. Subprocess is a built in module,  you should not need to install it.  Simply ```import python```

**subprocess allows you to run shell commands from python**

In [ ]:
import subprocess

subprocess.check_output(...) allows you to run a command. It returns the output as bytes.

In [31]:
subprocess.check_output(["ls", "/"])

b'bin\nboot\ndev\netc\nhome\nlib\nlib64\nlost+found\nmedia\nmnt\nnotebooks\nopt\nproc\nrivendell\nroot\nrun\nsbin\nsqldata\nsrv\nsys\ntmp\nusr\nvar\n'

### A few things to note:

+ In Python 3 b'...' means the return value is bytes. Python 2 assumed that the return value is a string. This leads to all kinds of problems if what you're working with are actually bytes.
+ The output contains a few special characters, specifically '\n'. '\n'  means "carriage return" and you can treat it like a normal character in Python.

Python makes it is easy to convrt bytes into a string (assuming the bytes are a string!). To do so we can add .decode(...) 

In [32]:
subprocess.check_output(["ls", "/"]).decode('utf-8')

'bin\nboot\ndev\netc\nhome\nlib\nlib64\nlost+found\nmedia\nmnt\nnotebooks\nopt\nproc\nrivendell\nroot\nrun\nsbin\nsqldata\nsrv\nsys\ntmp\nusr\nvar\n'

We can split up the output 

In [33]:
subprocess.check_output(["ls", "/"]).decode('utf-8').split("\n")

['bin',
 'boot',
 'dev',
 'etc',
 'home',
 'lib',
 'lib64',
 'lost+found',
 'media',
 'mnt',
 'notebooks',
 'opt',
 'proc',
 'rivendell',
 'root',
 'run',
 'sbin',
 'sqldata',
 'srv',
 'sys',
 'tmp',
 'usr',
 'var',
 '']

In [34]:
subprocess.check_output(["fib", "20"]).decode('utf-8')

"The fibonacci number for '20' is 6765\n"

In [41]:
subprocess.check_output(["fib", "-l", "20"]).decode('utf-8').split("\n")

["The fibonacci number for '0' is 0",
 "The fibonacci number for '1' is 1",
 "The fibonacci number for '2' is 1",
 "The fibonacci number for '3' is 2",
 "The fibonacci number for '4' is 3",
 "The fibonacci number for '5' is 5",
 "The fibonacci number for '6' is 8",
 "The fibonacci number for '7' is 13",
 "The fibonacci number for '8' is 21",
 "The fibonacci number for '9' is 34",
 "The fibonacci number for '10' is 55",
 "The fibonacci number for '11' is 89",
 "The fibonacci number for '12' is 144",
 "The fibonacci number for '13' is 233",
 "The fibonacci number for '14' is 377",
 "The fibonacci number for '15' is 610",
 "The fibonacci number for '16' is 987",
 "The fibonacci number for '17' is 1597",
 "The fibonacci number for '18' is 2584",
 "The fibonacci number for '19' is 4181",
 "The fibonacci number for '20' is 6765",
 '']

In [ ]:
p = subprocess.Popen(["fib", "40"])